In [3]:
from yahoo_fin import stock_info as si


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
stocks_ticker = pd.read_csv("stocks_ticker.csv")
base_url = "https://ticker.finology.in/company/"

In [3]:
#Creating list of Stocks listed in NSE
def get_stock_names():
    stocks = list(stocks_ticker['Company Name'])
    return stocks

#Creating dictionary of stocks and their tickers for fetching data from yahoo finance
def get_stock_ticker_dict():
    dict_stocks = dict(zip(stocks_ticker['Company Name'],stocks_ticker.Symbol))
    return dict_stocks

#Fundamental Analysis

#Getting Financial Tables
def get_tables(ticker):
    tables = pd.read_html("https://ticker.finology.in/company/"+ticker)
    return tables

#Separating Balance Sheet
def get_balance_sheet(ticker):
    balance_sheet = get_tables(ticker)[3]
    balance_sheet.reset_index(drop=True,inplace=True)
    balance_sheet.fillna("-",inplace=True)
    return balance_sheet

#Separating Profit and Loss Statement
def get_profit_and_loss(ticker):
    p_and_l = get_tables(ticker)[2]
    return p_and_l

#Separating Cash Flow Statement
def get_cashflow(ticker):
    cash_flow = get_tables(ticker)[4]
    cash_flow = cash_flow.iloc[:,0:6]
    return cash_flow

#Separating Quarterly Results
def get_quarterly_results(ticker):
    quarter_results = get_tables(ticker)[1]
    return quarter_results

#Separating Promoter Details
def get_promoter_details(ticker):
    promoters = get_tables(ticker)[5]
    return promoters

#Separating Promoter Details in case of Banking Stocks
def get_promoter_details_bank(ticker):
    promoters = get_tables(ticker)[4]
    return promoters

#Separating Investor Details
def get_investor_details(ticker):
    investors = get_tables(ticker)[6]
    return investors

#Separating Investor Details in case of Banking stocks
def get_investor_details_bank(ticker):
    investors = get_tables(ticker)[5]
    return investors

#Separating Pledging Details
def get_promoter_pledging(ticker):
    pledging = get_tables(ticker)[0]
    return pledging

#Technical Analysis

#Simple Moving Average
def SMA(data, ndays): 
    SMA = pd.Series(data['close'].rolling(ndays).mean(), name = 'SMA') 
    data = data.join(SMA) 
    return data

#Exponentially-weighted Moving Average 
def EWMA(data, ndays): 
    EMA = pd.Series(data['close'].ewm(span = ndays, min_periods = ndays - 1).mean(), 
                 name = 'EWMA_' + str(ndays)) 
    data = data.join(EMA) 
    return data

#Calculating Simple Moving Average for given duration and plotting it
def calculate_and_plot__sma(data,ndays,ticker):
    sma = SMA(data,ndays)
    sma = sma.dropna()
    sma = sma['SMA']
    temp_df = pd.DataFrame({"Price":data["close"],"SMA":sma},index=data.index)
    fig = px.line(temp_df,x=temp_df.index,y=temp_df.columns,title=ticker, labels={'value': 'Price', 'index': 'Date'},line_shape="spline")
    return fig

#Calculating Exponentially-weighted Moving Average for given duration and plotting it
def calculate_and_plot__ewma(data,ndays,ticker):
    ewma = EWMA(data,ndays)
    ewma = ewma.dropna()
    ewma = ewma['EWMA_'+str(ndays)]
    temp_df = pd.DataFrame({"Price":data["close"],"EWMA":ewma},index=data.index)
    fig = px.line(temp_df,x=temp_df.index,y=temp_df.columns,title=ticker, labels={'value': 'Price', 'index': 'Date'},line_shape="spline")
    return fig

#Relative-Strength Index
def rsi(close, periods = 14):
    close_delta = close.diff()
    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

#Calculating RSI and plotting it
def calculate_and_plot_rsi(data,ticker):
    data['RSI'] = rsi(data['close'])
    temp_df = pd.DataFrame({"Price":data['close'],"RSI":data['RSI']},index=data.index)
    fig = px.line(temp_df, x=temp_df.index, y=temp_df.RSI, title=ticker, labels={'value': 'Price', 'index': 'Date'},line_shape="spline")
    return fig

#Bollinger Bands
def bollinger_bands(data,n):
    MA = data.close.rolling(window=n).mean()
    SD = data.close.rolling(window=n).std()
    data['MiddleBand'] = MA
    data['UpperBand'] = MA + (2 * SD) 
    data['LowerBand'] = MA - (2 * SD)
    return data

#Calculating Bollinger Bands and plotting it
def calculate_and_plot_bollinger(data,ndays,ticker):
    data = bollinger_bands(data,ndays)
    temp_df = pd.DataFrame({"Price":data['close'],"Upper Band":data['UpperBand'],"Middle Band":data["MiddleBand"],"Lower Band":data['LowerBand']},index=data.index)
    fig = px.line(temp_df,x=temp_df.index,y=temp_df.columns,title=ticker,labels={'value': 'Price', 'index': 'Date'},color_discrete_map={"Upper Band": "black", "Middle Band": "red", "Lower Band": "black","Price":"blue"})
    return fig
    
#Volume
def plot_volume(data,ticker):
    fig = px.bar(data,x=data.index,y=data['volume']/1000000,title=ticker,labels={'y':'Million','index':'Date'})
    return fig


In [15]:
a = 'jayant'

In [17]:
a[:-3]

'jay'

In [24]:
def build_url(ticker, start_date = None, end_date = None, interval = "1d"):
    base_url = "https://query1.finance.yahoo.com/v8/finance/chart/"
    
    if end_date is None:  
        end_seconds = int(pd.Timestamp("now").timestamp())
        
    else:
        end_seconds = int(pd.Timestamp(end_date).timestamp())
        
    if start_date is None:
        start_seconds = 7223400    
        
    else:
        start_seconds = int(pd.Timestamp(start_date).timestamp())
    
    site = base_url + ticker
    
    params = {"period1": start_seconds, "period2": end_seconds,
              "interval": interval.lower(), "events": "div,splits"}
    
    
    return site, params

def get_data(ticker, start_date = None, end_date = None, index_as_date = True,
             interval = "1d", headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
):
    '''Downloads historical stock price data into a pandas data frame.  Interval
       must be "1d", "1wk", "1mo", or "1m" for daily, weekly, monthly, or minute data.
       Intraday minute data is limited to 7 days.
    
       @param: ticker
       @param: start_date = None
       @param: end_date = None
       @param: index_as_date = True
       @param: interval = "1d"
    '''
    
    if interval not in ("1d", "1wk", "1mo", "1m"):
        raise AssertionError("interval must be of of '1d', '1wk', '1mo', or '1m'")
    
    
    # build and connect to URL
    site, params = build_url(ticker, start_date, end_date, interval)
    resp = requests.get(site, params = params, headers = headers)
    
    
    if not resp.ok:
        raise AssertionError(resp.json())
        
    
    # get JSON response
    data = resp.json()
    
    # get open / high / low / close data
    frame = pd.DataFrame(data["chart"]["result"][0]["indicators"]["quote"][0])

    # get the date info
    temp_time = data["chart"]["result"][0]["timestamp"]

    if interval != "1m":
    
        # add in adjclose
        frame["adjclose"] = data["chart"]["result"][0]["indicators"]["adjclose"][0]["adjclose"]   
        frame.index = pd.to_datetime(temp_time, unit = "s")
        frame.index = frame.index.map(lambda dt: dt.floor("d"))
        frame = frame[["open", "high", "low", "close", "adjclose", "volume"]]
            
    else:

        frame.index = pd.to_datetime(temp_time, unit = "s")
        frame = frame[["open", "high", "low", "close", "volume"]]
        
        
    frame['ticker'] = ticker.upper()
    
    if not index_as_date:  
        frame = frame.reset_index()
        frame.rename(columns = {"index": "date"}, inplace = True)
        
    return frame

def force_float(elt):
    
    try:
        return float(elt)
    except:
        return elt
    
def get_live_price(ticker):
    
    '''Gets the live price of input ticker
    
       @param: ticker
    '''    
    
    df = get_data(ticker, end_date = pd.Timestamp.today() + pd.DateOffset(10))
    
    
    return df.close[-1]

In [29]:
def get_quote_table(ticker , dict_result = True, headers = {'User-agent': 'Mozilla/5.0'}): 
    
    '''Scrapes data elements found on Yahoo Finance's quote page 
       of input ticker
    
       @param: ticker
       @param: dict_result = True
    '''

    site = "https://finance.yahoo.com/quote/" + ticker + "?p=" + ticker
    
    tables = pd.read_html(StringIO(requests.get(site, headers=headers).text))

    data = pd.concat([tables[0], tables[1]], axis=0)

    data.columns = ["attribute" , "value"]
    
    quote_price = pd.DataFrame(["Quote Price", get_live_price(ticker)]).transpose()
    quote_price.columns = data.columns.copy()
    
    data = pd.concat([data, quote_price], axis=0)
    
    data = data.sort_values("attribute")
    
    data = data.drop_duplicates().reset_index(drop = True)
    
    data["value"] = data.value.map(force_float)

    if dict_result:
        
        result = {key : val for key,val in zip(data.attribute , data.value)}
        return result
        
    return data 

In [6]:
stocks = get_stock_names()
stock_dict = get_stock_ticker_dict()

In [1]:
import requests
from io import StringIO
import pandas as pd

In [89]:
response = requests.get("https://ticker.finology.in/company/TCS")

In [90]:
from bs4 import BeautifulSoup

# Assuming the HTML content is stored in a variable called 'html_content'
soup = BeautifulSoup(response.text, 'html.parser')

In [125]:
values_dict = {}

In [126]:
values_dict['Quote Price'] = soup.select_one('span.Number').text.strip()
values_dict["high"] = soup.select_one('span#mainContent_ltrlTodayHigh').text.strip()
values_dict["low"] = soup.select_one('span#mainContent_ltrlTodayLow').text.strip()
values_dict["week_52_high"] = soup.select_one('span#mainContent_ltrl52WH').text.strip()
values_dict["week_52_low"] = soup.select_one('span#mainContent_ltrl52WL').text.strip()

In [92]:
div_content = soup.select_one('div#mainContent_updAddRatios')

In [127]:
for i in div_content.find_all('div', 'col-6 col-md-4 compess'):
	try:
		values_dict[i.find('small').text.strip()] = i.find('p').text.strip().replace("\n","").replace("\r","").replace(" ","").replace("\xa0", "")
	except:
		continue

In [128]:
values_dict

{'price': '3960.20',
 'high': '3971',
 'low': '3910.05',
 'week_52_high': '4254.75',
 'week_52_low': '3156',
 'Market Cap': '₹1432835.02Cr.',
 'Enterprise Value': '₹1426236.02Cr.',
 'No. of Shares': '361.81Cr.',
 'P/E': '32.89',
 'P/B': '19.96',
 'Face Value': '₹1',
 'Div. Yield': '1.84%',
 'Book Value (TTM)': '₹198.39',
 'CASH': '₹6599Cr.',
 'DEBT': '₹0Cr.',
 'Promoter Holding': '71.77%',
 'EPS (TTM)': '₹120.39',
 'Sales Growth': '6.31%',
 'ROE': '59.6%',
 'ROCE': '79.73%',
 'Profit Growth': '11.39%'}

In [181]:
import requests
import random
import string

# Define a function to generate a random User-Agent

    # Example list of User-Agents
user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15"
    ]
random.choice(user_agents)

'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'

In [182]:
response = requests.get("https://finance.yahoo.com/quote/INFY.NS?p=INFY.NS", headers={'User-agent': random.choice(user_agents)})
# html_string = StringIO(response.text)
# tables = pd.read_html(html_string)
# data = pd.concat([tables[0], tables[1]], axis=0)

In [185]:
response = requests.get("https://www.google.com/finance/quote/INFY:NSE", headers={'User-agent': random.choice(user_agents)})

In [188]:
soup_google = BeautifulSoup(response.content, 'html.parser')

In [190]:
soup_google.find('div', 'YMlKec fxKbKc').text

'₹1,434.15'

In [196]:
data_google = soup_google.find_all("div", class_="gyFHrc")

In [200]:
for i in data_google:
	print("Name :", i.find("div", class_="mfs7Fc").text)
	print("Value :", i.find("div", class_="P6K39c").text)

Name : Previous close
Value : ₹1,427.30
Name : Day range
Value : ₹1,424.85 - ₹1,449.95
Name : Year range
Value : ₹1,239.05 - ₹1,733.00
Name : Market cap
Value : 5.96T INR
Name : Avg Volume
Value : 7.85M
Name : P/E ratio
Value : 22.66
Name : Dividend yield
Value : 2.65%
Name : Primary exchange
Value : NSE
Name : CEO
Value : Salil Parekh
Name : Founded
Value : Jul 2, 1981
Name : Headquarters
Value : Bengaluru, KarnatakaIndia
Name : Website
Value : infosys.com
Name : Employees
Value : 317,240


In [207]:
def get_quote_table_google(ticker):
    site = "https://www.google.com/finance/quote/" + ticker +":NSE"
    response = requests.get(site, headers={'User-agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.content, 'html.parser')
    values_dict = {}
    values_dict["Quote Price"] = soup.find('div', 'YMlKec fxKbKc').text
    data = soup.find_all("div", class_="gyFHrc")
    for i in data:
        values_dict[i.find("div", class_="mfs7Fc").text] = i.find("div", class_="P6K39c").text.strip()
    return values_dict

In [208]:
get_quote_table_google("INFY")

{'Quote Price': '₹1,434.15',
 'Previous close': '₹1,427.30',
 'Day range': '₹1,424.85 - ₹1,449.95',
 'Year range': '₹1,239.05 - ₹1,733.00',
 'Market cap': '5.96T INR',
 'Avg Volume': '7.85M',
 'P/E ratio': '22.66',
 'Dividend yield': '2.65%',
 'Primary exchange': 'NSE',
 'CEO': 'Salil Parekh',
 'Founded': 'Jul 2, 1981',
 'Headquarters': 'Bengaluru, KarnatakaIndia',
 'Website': 'infosys.com',
 'Employees': '317,240'}

In [156]:
def get_quote_table(ticker , dict_result = True, headers = {'User-agent': 'Mozilla/5.0'}): 
    site = "https://finance.yahoo.com/quote/" + ticker + "?p=" + ticker
    response = requests.get(site, headers={'User-agent': 'Mozilla/5.0'})
    soup2 = BeautifulSoup(response.content, 'html.parser')
    values_dict = {}
    values_dict["Quote Price"] = soup2.find('fin-streamer', class_='livePrice')['data-value']
    data = soup2.find_all("li","svelte-tx3nkj")
    for i in data:
	    values_dict[i.find("span","label svelte-tx3nkj").text] = i.find("span","value svelte-tx3nkj").text.strip()
    return values_dict

In [184]:
get_quote_table("TCS.NS")

{'Quote Price': '3960.2',
 'Previous Close': '3,976.95',
 'Open': '3,962.00',
 'Bid': '--',
 'Ask': '--',
 "Day's Range": '3,910.05 - 3,971.00',
 '52 Week Range': '3,156.00 - 4,254.75',
 'Volume': '1,726,442',
 'Avg. Volume': '2,523,752',
 'Market Cap (intraday)': '14.328T',
 'Beta (5Y Monthly)': '0.52',
 'PE Ratio (TTM)': '32.23',
 'EPS (TTM)': '122.89',
 'Earnings Date': 'Jul 11, 2024',
 'Forward Dividend & Yield': '112.00 (2.82%)',
 'Ex-Dividend Date': 'May 16, 2024',
 '1y Target Est': '4,186.40'}

In [150]:
values_dict = {}

In [175]:
soup2 = BeautifulSoup(response.content, 'html.parser')

values_dict["Quote Price"] = soup2.find('fin-streamer', class_='livePrice')['data-value']
data = soup2.find_all("li","svelte-tx3nkj")
for i in data:
	values_dict[i.find("span","label svelte-tx3nkj").text] = i.find("span","value svelte-tx3nkj").text.strip()

TypeError: 'NoneType' object is not subscriptable

In [152]:
values_dict

{'Quote Price': '3960.2',
 'Previous Close': '3,976.95',
 'Open': '3,962.00',
 'Bid': '--',
 'Ask': '--',
 "Day's Range": '3,910.05 - 3,971.00',
 '52 Week Range': '3,156.00 - 4,254.75',
 'Volume': '1,726,442',
 'Avg. Volume': '2,523,752',
 'Market Cap (intraday)': '14.328T',
 'Beta (5Y Monthly)': '0.52',
 'PE Ratio (TTM)': '32.23',
 'EPS (TTM)': '122.89',
 'Earnings Date': 'Jul 11, 2024',
 'Forward Dividend & Yield': '112.00 (2.82%)',
 'Ex-Dividend Date': 'May 16, 2024',
 '1y Target Est': '4,186.40'}

In [30]:
ticker = stock_dict["ICICI Bank Limited"]
ticker_yf = ticker+".NS"
quote_data = get_quote_table(ticker_yf)

C:\Users\91879\AppData\Local\Temp\ipykernel_8044\4093654230.py:99: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]


In [31]:
quote_data

{'1y Target Est': 788.07,
 '52 Week Range': '796.00 - 1,043.70',
 'Ask': '0.00 x 0',
 'Avg. Volume': 13613257.0,
 'Beta (5Y Monthly)': 0.8,
 'Bid': '0.00 x 0',
 "Day's Range": '1,004.60 - 1,015.80',
 'EPS (TTM)': 56.4,
 'Earnings Date': 'Jan 20, 2024',
 'Ex-Dividend Date': 'Aug 09, 2023',
 'Forward Dividend & Yield': '8.00 (0.80%)',
 'Market Cap': '7.078T',
 'Open': 1006.1,
 'PE Ratio (TTM)': 17.89,
 'Previous Close': 1009.85,
 'Quote Price': 1009.0499877929688,
 'Volume': 11503865.0}

In [ ]:
def get_tables(ticker):
    tables = pd.read_html("https://ticker.finology.in/company/"+"ICICIBANK")
    return tables

#Separating Balance Sheet
def get_balance_sheet(ticker):
    balance_sheet = get_tables(ticker)[3]
    balance_sheet.reset_index(drop=True,inplace=True)
    balance_sheet.fillna("-",inplace=True)
    return balance_sheet

In [34]:
res = requests.get("https://ticker.finology.in/company/"+"ICICIBANK").text
tables = pd.read_html(StringIO(res))

In [36]:
tables[3]

,Particulars,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023
0,Equity and Liabilities,NaN,NaN,NaN,NaN,NaN
1,Share Capital,1289.46,1294.76,1383.41,1389.97,1396.78
2,Total Reserves,107073.91,115206.16,146122.67,168855.59,198557.72
3,Deposits,652919.67,770968.99,932522.16,1064571.61,1180840.70
4,Borrowings,165319.97,162896.76,91630.96,107231.36,119325.49
5,Other Liabilities,37851.46,47994.99,58770.37,68982.79,83325.08
6,Total Liabilities,964459.15,1098365.15,1230432.68,1411297.74,1584206.65
7,Assets,NaN,NaN,NaN,NaN,NaN
8,Balance with RBI,37858.01,35283.96,46031.19,109522.82,68526.17
9,Balance with Banks,42438.27,83871.78,87097.06,58299.54,50912.10


In [161]:
ticker = "TCS.NS"
headers = {'User-agent': 'Mozilla/5.0'}

In [162]:
stats_site = "https://finance.yahoo.com/quote/" + ticker + \
                 "/key-statistics?p=" + ticker
    

tables = pd.read_html(StringIO(requests.get(stats_site, headers=headers).text))

tables = [table for table in tables if "Trailing P/E" in table.iloc[:,0].tolist()]


table = tables[0].reset_index(drop = True)

In [169]:
table = pd.DataFrame(table)

In [170]:
table.fillna("n/a",inplace=True)
attr = list(table.iloc[:,0])
val = list(table.iloc[:,1])
stats_value_dict = dict(zip(attr,val))

In [171]:
stats_value_dict

{'Market Cap': '14.39T',
 'Enterprise Value': '13.92T',
 'Trailing P/E': '32.41',
 'Forward P/E': '26.81',
 'PEG Ratio (5yr expected)': '2.83',
 'Price/Sales': '6.09',
 'Price/Book': '14.29',
 'Enterprise Value/Revenue': '5.78',
 'Enterprise Value/EBITDA': '20.54'}

In [172]:
stats_value_dict['PEG Ratio (5 yr expected)']

KeyError: 'PEG Ratio (5 yr expected)'